### 필수과제3
- sales 데이터를 드릴 예정
- 해당 세일즈 데이터 중에서 가장 많이 판매되었던 상품을 1, 2, 3, 4까지 라벨링해 주시고 해당 라벨링 값을 정답으로 하여 
- 위에 배운 내용을 복습하는 코드를 작성해 주세요!

- 해당 데이터는 정답값이 없다. 우리가 정답값을 만들어야 한다.
- 그 정답값을 만들어서 Decisiontree 모델만 사용하여 진행
- skf를 이용해서 동일하게 micro, macro, weighted를 비교하여 정리해 주시고
- 데이터의 전처리의 경우 시간이 오래 걸릴 수 있으니 최대한 시간 효율보셔서 진행해 주시면 된다.
- 다만 데이터 전처리가 없으면 안 됩니다.
- 1,2,3,4 외에 나머지 데이터는 0로 해주시고 데이터 전체 삭제하지마시고 분류해 주세요!
- 1,2,3,4,0 라벨값이 생성될 것

In [2]:
import pandas as pd
import numpy as np

In [10]:
sales_df=pd.read_csv('C:/open/sales_data.csv',parse_dates=['InvoiceDate'],encoding='unicode_escape')
sales=sales_df.copy()       # 데이터 복사

print(sales.dtypes)               # 데이터 확인
print(sales.shape)
print(sales.describe())

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object
(541909, 8)
            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000   16791.000000
max     80995.000000   38970.000000   18287.000000


In [32]:
sales

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [47]:
# 객체 데이터를 골라서 범주형으로 전환
sales.loc[:,sales.dtypes=='object']=sales.select_dtypes(['object']).apply(lambda x:x.astype('category'))

salesNum=sales.select_dtypes(include=['number'])
print(salesNum.info())
#salesCat=sales.select_dtypes(include='category')
print(salesCat.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540455 entries, 0 to 541908
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Quantity    540455 non-null  int64  
 1   UnitPrice   540455 non-null  float64
 2   CustomerID  406829 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 32.6 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 540455 entries, 0 to 541908
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   InvoiceNo    540455 non-null  category
 1   StockCode    540455 non-null  category
 2   Description  540455 non-null  category
 3   Country      540455 non-null  category
dtypes: category(4)
memory usage: 24.9 MB
None


In [31]:
# 인덱스
print(sales.index)
print(sales.index.unique())         # 전체 데이터는 540455개 인데 왜 인덱스는 541908까지만?

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            541899, 541900, 541901, 541902, 541903, 541904, 541905, 541906,
            541907, 541908],
           dtype='int64', length=540455)
Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            541899, 541900, 541901, 541902, 541903, 541904, 541905, 541906,
            541907, 541908],
           dtype='int64', length=540455)


In [11]:
sales.isna().sum()      # 결측치 확인

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [13]:
sales.dropna(subset=['Description'],inplace=True)
sales.isna().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133626
Country             0
dtype: int64

In [28]:
salesNaN=sales.loc[sales.CustomerID.isna()]
salesNaN.StockCode.value_counts()               # CustomerID가 없는 데이터는 어떻게 처리하면 좋을까?
                                                # CustomerID가 없는 데이터끼리 연관성이 있는가?

DOT       693
85099B    497
21931     414
22411     388
20712     372
         ... 
35598C      1
35817P      1
35607B      1
85230E      1
84859B      1
Name: StockCode, Length: 3482, dtype: int64

In [54]:
sales['StockCode'].nunique()     # 고유한 제품 코드 갯수->4070종류의 제품을 판매

4070

In [66]:
# 많이 팔린 제품 찾기
sales.groupby('StockCode').sum().sort_values(by=['Quantity'],ascending=False)

C:\Users\KYW\AppData\Local\Temp\ipykernel_7204\3254344606.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sales.groupby('StockCode').sum().sort_values(by=['Quantity'],ascending=False)


,Quantity,UnitPrice,CustomerID
StockCode,,,
22197,56450,1527.11,17275441.0
84077,53847,173.39,7297844.0
85099B,47363,5333.30,25396253.0
85123A,38830,7177.79,32318742.0
84879,36221,2584.77,21602517.0
...,...,...,...
79323LP,-2618,0.00,0.0
79323W,-4838,6.75,15831.0
72140F,-5368,0.00,0.0


>많이 팔린 상위 4개 품목: 22197, 84077, 85099B, 85123A

In [67]:
# 라벨링
target = []
for code in sales['StockCode']:
    if code == '22197':
        target.append(1)
    elif code == '84077':
        target.append(2)
    elif code == '85099B':
        target.append(3)
    elif code == '85123A':
        target.append(4)
    else:
        target.append(0)

In [76]:
sales['target']=target
sales.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,target
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom,0
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom,0
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,0
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,0
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom,0


In [90]:
sales['Country_Code']=sales['Country'].factorize()[0]
sales.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,target,Country_Code
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,0,0
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,0,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0,0
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom,0,0
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom,0,0
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,0,0
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,0,0
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom,0,0


In [109]:
feature=sales[['Quantity','UnitPrice','Country_Code']].values      # 날짜 데이터도 뽑아서 사용하고 싶은데 잘 모르겠습니다..
label=sales['target'].values

- Straitified KFold 사용

In [100]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score

- macro: precision, recall 매우 낮음

In [108]:
result_skfold = StratifiedKFold(n_splits=5)
result_clf = DecisionTreeClassifier(random_state=40)
n_iter = 0 

cv_accuracy=[]
cv_precision=[]
cv_recall=[]


for train_idx, test_idx in result_skfold.split(feature, label):
    
    X_train, X_test = feature[train_idx], feature[test_idx]    # 훈련셋, 테스트셋 분리
    y_train, y_test = label[train_idx], label[test_idx]
    
    result_clf.fit(X_train, y_train)  # 모델 훈련
    
    pred = result_clf.predict(X_test)  # 예측 
    
    #정확도 및 정밀도, 재현율도 검사
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, pred),3)  #정확도
    precision = np.round(precision_score(y_test,pred, average='macro'), 3)  #정밀도, 라벨이 5종류라 average 매개변수추가(이진이 아닐시 문제 발생)
    recall = np.round(recall_score(y_test, pred, average='macro'), 3)   #재현율, 라벨이 5종류라 average 매개변수추가(이진이 아닐시 문제 발생)

    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print('\n 평균 검증 정확도', np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))

c:\Users\KYW\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



1 교차검증 정확도 : 0.988, 교차검증 precision :0.198, 교차검증 recall :0.2

2 교차검증 정확도 : 0.988, 교차검증 precision :0.198, 교차검증 recall :0.2

3 교차검증 정확도 : 0.988, 교차검증 precision :0.198, 교차검증 recall :0.2

4 교차검증 정확도 : 0.987, 교차검증 precision :0.198, 교차검증 recall :0.2

5 교차검증 정확도 : 0.987, 교차검증 precision :0.22, 교차검증 recall :0.202

 평균 검증 정확도 0.9875999999999999 0.2024 0.2004


- micro

In [110]:
result_skfold = StratifiedKFold(n_splits=5)
result_clf = DecisionTreeClassifier(random_state=40)
n_iter = 0 

cv_accuracy=[]
cv_precision=[]
cv_recall=[]


for train_idx, test_idx in result_skfold.split(feature, label):
    
    X_train, X_test = feature[train_idx], feature[test_idx]    # 훈련셋, 테스트셋 분리
    y_train, y_test = label[train_idx], label[test_idx]
    
    result_clf.fit(X_train, y_train)  # 모델 훈련
    
    pred = result_clf.predict(X_test)  # 예측 
    
    #정확도 및 정밀도, 재현율도 검사
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, pred),3)  #정확도
    precision = np.round(precision_score(y_test,pred, average='micro'), 3)  #정밀도, 라벨이 5종류라 average 매개변수추가(이진이 아닐시 문제 발생)
    recall = np.round(recall_score(y_test, pred, average='micro'), 3)   #재현율, 라벨이 5종류라 average 매개변수추가(이진이 아닐시 문제 발생)

    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print('\n 평균 검증 정확도', np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 : 0.988, 교차검증 precision :0.988, 교차검증 recall :0.988

2 교차검증 정확도 : 0.988, 교차검증 precision :0.988, 교차검증 recall :0.988

3 교차검증 정확도 : 0.988, 교차검증 precision :0.988, 교차검증 recall :0.988

4 교차검증 정확도 : 0.987, 교차검증 precision :0.987, 교차검증 recall :0.987

5 교차검증 정확도 : 0.987, 교차검증 precision :0.987, 교차검증 recall :0.987

 평균 검증 정확도 0.9875999999999999 0.9875999999999999 0.9875999999999999


- weighted

In [111]:
result_skfold = StratifiedKFold(n_splits=5)
result_clf = DecisionTreeClassifier(random_state=40)
n_iter = 0 

cv_accuracy=[]
cv_precision=[]
cv_recall=[]


for train_idx, test_idx in result_skfold.split(feature, label):
    
    X_train, X_test = feature[train_idx], feature[test_idx]    # 훈련셋, 테스트셋 분리
    y_train, y_test = label[train_idx], label[test_idx]
    
    result_clf.fit(X_train, y_train)  # 모델 훈련
    
    pred = result_clf.predict(X_test)  # 예측 
    
    #정확도 및 정밀도, 재현율도 검사
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, pred),3)  #정확도
    precision = np.round(precision_score(y_test,pred, average='weighted'), 3)  #정밀도, 라벨이 5종류라 average 매개변수추가(이진이 아닐시 문제 발생)
    recall = np.round(recall_score(y_test, pred, average='weighted'), 3)   #재현율, 라벨이 5종류라 average 매개변수추가(이진이 아닐시 문제 발생)

    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print('\n 평균 검증 정확도', np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))

c:\Users\KYW\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



1 교차검증 정확도 : 0.988, 교차검증 precision :0.976, 교차검증 recall :0.988

2 교차검증 정확도 : 0.988, 교차검증 precision :0.976, 교차검증 recall :0.988

3 교차검증 정확도 : 0.988, 교차검증 precision :0.976, 교차검증 recall :0.988

4 교차검증 정확도 : 0.987, 교차검증 precision :0.976, 교차검증 recall :0.987

5 교차검증 정확도 : 0.987, 교차검증 precision :0.976, 교차검증 recall :0.987

 평균 검증 정확도 0.9875999999999999 0.976 0.9875999999999999
